[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# DIY Covid-19 Dashboard

This is a template for your DIY Covid Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the EECS JupyterHub as well as in the binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [32]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [33]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data (the code below will be hidden when the dashboard is rendered by Voila).

In [34]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
jsondata={'aVb' : 'bedcomparison.json'}

## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and generate a dataframe with some random data

In [30]:
def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    df=pd.DataFrame(index=range(0,100), columns=['One', 'Two'])
    # we have no real data to wrangle, so we just generate two random walks.
    one=two=0.0
    for i in range(0,100):
        df.loc[i,'One']=one
        df.loc[i,'Two']=two
        one+=np.random.randn()
        two+=2*np.random.randn()
    return df

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
# df=wrangle_data() # df is the dataframe for plotting

In [37]:
def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    with open("bedcomparison.json", "rt") as INFILE:
        data=json.load(INFILE)
    datalist=data['data']
    dates=[dictionary['Date'] for dictionary in datalist ]
    dates.sort()
    startdate= pd.to_datetime(dates[0], format="%Y-%m-%d")
    enddate= pd.to_datetime(dates[-1], format="%Y-%m-%d")
    index=pd.date_range(startdate, enddate, freq='D')
    bedOccupancydf=pd.DataFrame(index=index, columns=["Admissions", "Occupancybed"])
    for entry in datalist: 
        date=pd.to_datetime(entry['Date'], format="%Y-%m-%d")
        for column in ['Admissions', 'Occupancybed']:
            if pd.isna(bedOccupancydf.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                bedOccupancydf.loc[date, column]=value

    bedOccupancydf.fillna(0.0, inplace=True)
    return bedOccupancydf
# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
df = wrangle_data(jsondata['aVb']) 
df = df[0:100]

## Download current data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting;
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

Clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [49]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = [
        'areaType=nation',
        'areaName=England'
    ]

    structure = {
    "Date": "date",
    "Cumulativecases": "cumCasesByPublishDateRate",
    "Population": "VaccineRegisterPopulationByVaccinationDate",
    }
    apivbed = Cov19API(filters=filters, structure=structure)
    bed=apibed.get_json()
    return bed # return data read from the API

In [50]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    changeWidget()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Refresh data', # you may want to change this...
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Keep calm and carry on",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='exclamation-triangle'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(description='Refresh data', icon='exclamation-triangle', style=ButtonStyle(), tooltip='Keep calm and ca…

## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [20]:
Walesage_df=pd.read_pickle("Walesage_df.pkl")

In [ ]:
def age_graph(graphcolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        Walesage_df.plot(kind='bar', y=(graphcolumns)) # graphcolumns is a tuple - we need a list
        plt.show() # important - graphs won't update properly if this is missing


# This code can be made better by adding a named parameter so the python interpreter can recognize the parameters when running the function. Therefore to make the code make the code run better, we have to modify it like this:

def age_graph(df,graphcolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        Walesage_df.plot(kind='bar', y=(graphcolumns)) # graphcolumns is a tuple - we need a list
        plt.show() # important - graphs won't update properly if this is missing


# In this case the "df" is the named parameter. 


In [54]:
(age_graph, {'graphcolumns': agecols})

# This code can be made simpler by simply changing to this (df = Walesage_df, graphcolumns = agecols)

(<function __main__.age_graph(graphcolumns)>,
 {'graphcolumns': Dropdown(description='Sex:', options=('males', 'females', 'total'), value='males')})

In [45]:
def age_graph(graphcolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        Walesage_df.plot(kind='bar', y=(graphcolumns)) # graphcolumns is a tuple - we need a list
        plt.show() # important - graphs won't update properly if this is missing
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
agecols=wdg.Dropdown(
    options=['males', 'females', 'total'],
    value='males',# can only take only one value in the form of a string.
    description='Sex:',
    disabled=False,
    
)

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=agecols.value
    if current==agecols.options[0]:
        other=agecols.options[1]
    elif current==agecols.options[1]:
        other=agecols.options[2]
    else:
        other=agecols.options[0]
    agecols.value=other # forces the redraw
    agecols.value=current # now we can change it back
    
    
graph=wdg.interactive_output(age_graph, {'graphcolumns': agecols})
    
display(agecols, graph)

Dropdown(description='Sex:', options=('males', 'females', 'total'), value='males')

Output()

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

In [53]:
 
series=wdg.SelectMultiple(
    options=['Admissions', 'Occupancybed'],
    value=['Admissions', 'Occupancybed'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    #    value='pineapple', # Defaults to 'pineapple'
    #    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

def changeWidget():
    tempScale = scale.value
    if scale.value == 'linear':
        scale.value = 'log'
    else:
        scale.value = 'linear'
    scale.value = tempScale

    # try replacing HBox with a VBox
controls=wdg.HBox([series, scale])

def admissions_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        df[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

    # keep calling admissions_graph(gcols=value_of_series, gscale=value_of_scale); 
    # capture output in widget graph   
graph=wdg.interactive_output( admissions_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)

Output()

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*